# Extraction of Frequency Bands and Asymetry Data


In [70]:
# Imports
import os
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
%matplotlib inline

### Create marker list
- Here the idea is to seperae the EEG data recorded for one participant into t(s) epochs (example t = 15s). 
- This yeilds more sampltes to train, validate and test our models

In [71]:
#Create marker list
def get_markers(epoch_length=15):
    sampling_rate = 125 # 125 Hz
    min_frac = (epoch_length/60.0)
    step_minutes = np.arange(0,5,min_frac) # Start - 0 min, total_recording_time - 5 min, step = 15s(i.e. 0.25 min)
    print(f"{step_minutes=}")
    step_minutes = np.expand_dims(step_minutes * sampling_rate * 60, axis=1)
    markers = np.concatenate( [step_minutes, np.zeros( step_minutes.shape ), np.ones( step_minutes.shape ) ], axis=1  ).astype(np.int64)
    return markers

epoch_length = 15 # This is 15 seconds, can be reduced to 10s if we need more data
markers = get_markers(epoch_length=epoch_length)
len(markers)

step_minutes=array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 ,
       2.75, 3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75])


20

### Convert to mne.io.RawArray

In [72]:
#Convert a Pandas df to mne.io.RawArray
def dataframe_to_raw(dataframe, sfreq):
    ch_names = list(dataframe.columns)
    ch_types = ['eeg'] * (len(dataframe.columns))
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    dataframe = dataframe.T  # mne looks at the tranpose() format
    dataframe *= 1e-6  # convert from uVolts to Volts (mne assumes Volts data)

    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq, verbose=False)

    raw = mne.io.RawArray(dataframe, info,verbose=False)
    raw.set_montage(ten_twenty_montage)
    
    return raw

### Read labels CSV

In [73]:
# Read the csv file with lables and put id, lable into a dict.
def read_labels(file_path):
    label_df = pd.read_csv(file_path)
    data = label_df[['No.', 'label']]
    data = data.to_numpy()
    dict_label = dict()

    for i in range(len(data)):
        dict_label[data[i][0]] = data[i][1]
    return dict_label

label_file_path = '../data/label_jo/PSS.csv'
dict_label = read_labels(label_file_path)    
print(dict_label)

{1: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 1, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 1, 35: 0, 36: 1, 37: 1, 38: 1, 39: 0, 40: 1, 41: 0, 42: 0, 43: 1, 44: 1, 45: 1, 46: 0, 47: 0, 48: 0, 49: 1, 50: 0, 51: 1, 52: 1, 53: 0, 54: 1, 55: 0}


In [74]:
file_name_dict = dict()
for i in range(len(dict_label)):
    num = i + 1
    file_name_dict[num] = str(num).zfill(3) + ".csv"
print(file_name_dict)

{1: '001.csv', 2: '002.csv', 3: '003.csv', 4: '004.csv', 5: '005.csv', 6: '006.csv', 7: '007.csv', 8: '008.csv', 9: '009.csv', 10: '010.csv', 11: '011.csv', 12: '012.csv', 13: '013.csv', 14: '014.csv', 15: '015.csv', 16: '016.csv', 17: '017.csv', 18: '018.csv', 19: '019.csv', 20: '020.csv', 21: '021.csv', 22: '022.csv', 23: '023.csv', 24: '024.csv', 25: '025.csv', 26: '026.csv', 27: '027.csv', 28: '028.csv', 29: '029.csv', 30: '030.csv', 31: '031.csv', 32: '032.csv', 33: '033.csv', 34: '034.csv', 35: '035.csv', 36: '036.csv', 37: '037.csv', 38: '038.csv', 39: '039.csv', 40: '040.csv', 41: '041.csv', 42: '042.csv', 43: '043.csv', 44: '044.csv', 45: '045.csv', 46: '046.csv', 47: '047.csv', 48: '048.csv', 49: '049.csv', 50: '050.csv', 51: '051.csv', 52: '052.csv', 53: '053.csv', 54: '054.csv', 55: '055.csv'}


### Export PSD featurs from all ch_16_csv files

In [75]:
#../data/data_jo/001.csv
temp_df = pd.read_csv('../data/data_jo/001.csv', dtype={'Marker':'object'})
temp_df = temp_df.head(37500)
temp_df.shape
#temp_df['Marker'].unique()


(37500, 18)

In [76]:
# Reference1 : https://github.com/akraradets/EEG_ChronicStress/blob/main/2-ChronicStress/3-export-feature.ipynb
# Reference2 : https://github.com/chaklam-silpasuwanchai/Python-for-EEG/blob/main/03%20-%20Asymmetry%20%2B%20DEAP.ipynb

def export_features_to_csv(in_dir_path, out_dir_path, file_name, participant_id, label):

    #dir_path = '../data/ch_16_csv'
    #f_name = 'conv_exp01.csv'
    f_path = in_dir_path + '/' + file_name
    print(f_path)
    # Read csv
    in_df = pd.read_csv(f_path, dtype={'Marker':'object'})
    in_df.drop(['Marker', 'timestamps'], axis=1, inplace=True)
    in_df = in_df.head(37500)
    

#     f_idx = file_name[3:]
#     f_idx = f_idx[:-4]
#     f_id = int(f_idx)
#     label = dict_label[f_id]
#     print(f_id, label)

    # df to mne.io.RawArray
    raw = dataframe_to_raw(in_df, 125)
    
    #Artifact removal
    raw.notch_filter(50) # 50 Hz from electrical network
    #Slow drift and high freq. removal
    raw.filter(1, 43)
    
    #Epochs
    epochs = mne.Epochs(raw, markers, tmin=0, tmax=8, baseline=None, verbose=False) #Our recording time is 6min thus as tmax 8min is used
    spectrum = epochs.compute_psd(method="welch", n_fft=125)  # Get Spectrum object from epochs

    #The shape of PSDs is (n_epochs,n_channels,n_freqs) and FREQs is (n_freqs)
    psds, freqs = spectrum.get_data(return_freqs=True) 
    # psds.shape, freqs.shape, (24, 16, 63) (63,)
    psds = 10 * np.log10(psds)

    band_names = np.array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta', 'Relative_gamma'])
    mean_band_names = np.array(['mean_Delta', 'mean_Theta', 'mean_Alpha', 'mean_Beta', 
                                'mean_Gamma', 'mean_Slow', 'mean_Low_beta', 'mean_Relative_gamma'])
    #filter_list = [[1,3],[4,7],[8,12],[13,30],[30,43], [4,13], [13,17]]

    #Take the mean of all channels
    mean_psds = psds.mean(axis=1)
    #mean_psds.shape, (24, 63)

    #Band mean extraction
    delta = mean_psds[:, 1:3]
    theta = mean_psds[:, 4:7]
    alpha = mean_psds[:, 8:12]
    beta  = mean_psds[:, 13:30]
    gamma = mean_psds[:, 30:43]
    slow = mean_psds[:, 4:13]
    low_beta = mean_psds[:, 13:17]

    #For each band take the mean of the power
    delta = delta.mean(axis=1).reshape(-1, 1)
    theta = theta.mean(axis=1).reshape(-1, 1)
    alpha = alpha.mean(axis=1).reshape(-1, 1)
    beta  = beta.mean(axis=1).reshape(-1, 1)
    gamma = gamma.mean(axis=1).reshape(-1, 1)
    slow = slow.mean(axis=1).reshape(-1, 1)
    low_beta = low_beta.mean(axis=1).reshape(-1, 1)
    relative_gamma = slow/gamma

    #Stack them together
    _temp = [delta, theta, alpha, beta, gamma, slow, low_beta, relative_gamma]
    all_bands = np.hstack(_temp)
    #all_bands.shape , (24, 7)

    out_df = pd.DataFrame(all_bands, columns = mean_band_names.tolist())

    # Jo's channels ['Fp1','Fp2','F3','F4','F7','F8','C3','C4','T3','T4','T5','T6','P3','P4','O1','O2']
    # Our channels['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']

    # Band extraction
    #channel_list = ['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']
    #Using Jo's band names
    channel_list = ['Fp1','Fp2','F3','F4','F7','F8','C3','C4','T3','T4','T5','T6','P3','P4','O1','O2']

    for i in range(len(channel_list)):
        new_band_names = []
        for band_name in band_names:
            new_name = channel_list[i] + '_' + band_name
            new_band_names.append(new_name)

        delta = psds[:, i, 1:3]
        theta = psds[:, i, 4:7]
        alpha = psds[:, i, 8:12]
        beta  = psds[:, i, 13:30]
        gamma = psds[:, i, 30:43]
        slow = psds[:, i, 4:13]
        low_beta = psds[:, i, 13:17]

        #For each band take the mean of the power
        delta = delta.mean(axis=1).reshape(-1, 1)
        theta = theta.mean(axis=1).reshape(-1, 1)
        alpha = alpha.mean(axis=1).reshape(-1, 1)
        beta  = beta.mean(axis=1).reshape(-1, 1)
        gamma = gamma.mean(axis=1).reshape(-1, 1)
        slow = slow.mean(axis=1).reshape(-1, 1)
        low_beta = low_beta.mean(axis=1).reshape(-1, 1)
        relative_gamma = slow/gamma

        #Stack them together
        _temp = [delta, theta, alpha, beta, gamma, slow, low_beta, relative_gamma]
        all_bands = np.hstack(_temp)
        temp_df = pd.DataFrame(all_bands, columns = new_band_names)
        out_df = pd.concat([out_df, temp_df], axis=1, join='inner')
    
    # alpha_frontal asymetry
    a_f4 = psds[:, 3, 8:12] # Jo's F4 index is 3
    a_f4 = a_f4.mean(axis=1).reshape(-1, 1)
    a_f3 = psds[:, 2, 8:12] # Jo's F3 index is 2
    a_f3 = a_f3.mean(axis=1).reshape(-1, 1)

    alpha_frontal = ((a_f4 - a_f3) / (a_f4 + a_f3)).reshape(1,-1)
    out_df['alpha_frontal'] = alpha_frontal[0]

    # alpha_temporal asymetry
    a_t4 = psds[:, 9, 8:12] # Jo's T4 index is 9
    a_t4 = a_t4.mean(axis=1).reshape(-1, 1)
    a_t3 = psds[:, 8, 8:12] # Jo's T3 index is 8
    a_t3 = a_t3.mean(axis=1).reshape(-1, 1)

    alpha_temporal = ((a_t4 - a_t3) / (a_t4 + a_t3)).reshape(1,-1)
    out_df['alpha_temporal'] = alpha_temporal[0]

    # alpha_asymmetry
    alpha_asymmetry = alpha_frontal[0] + alpha_temporal[0]
    out_df['alpha_asymmetry'] = alpha_asymmetry

    # beta_frontal asymetry
    b_f4 = psds[:, 3, 13:30] # Jo's F4 index is 3
    b_f4 = b_f4.mean(axis=1).reshape(-1, 1)
    b_f3 = psds[:, 2, 13:30] # Jo's F3 index is 2
    b_f3 = b_f3.mean(axis=1).reshape(-1, 1)

    beta_frontal = ((b_f4 - b_f3) / (b_f4 + b_f3)).reshape(1,-1)
    out_df['beta_frontal'] = beta_frontal[0]

    # beta_temporal asymetry
    b_t4 = psds[:, 9, 13:30] # Jo's T4 index is 9
    b_t4 = b_t4.mean(axis=1).reshape(-1, 1)
    b_t3 = psds[:, 8, 13:30] # Jo's T3 index is 8
    b_t3 = b_t3.mean(axis=1).reshape(-1, 1)

    beta_temporal = ((b_t4 - b_t3) / (b_t4 + b_t3)).reshape(1,-1)
    out_df['beta_temporal'] = beta_temporal[0]

    # beta_asymmetry
    beat_asymmetry = beta_frontal[0] + beta_temporal[0]
    out_df['beat_asymmetry'] = beat_asymmetry

    out_df['participant_id'] = participant_id
    out_df['label'] = label

    out_file_path = out_dir_path + '/' + file_name
    out_df.to_csv(out_file_path,index=False)


In [79]:
in_dir_path = '../data/data_jo'
out_dir_path = '../data/psd_filtered_features_csv_jo'

for i in range(len(dict_label)):
    num = i + 1
    participant_id = num
    label = dict_label[num]
    file_name = file_name_dict[num]
    #print(num, label, file_name)
    export_features_to_csv(in_dir_path, out_dir_path, file_name, participant_id, label)

../data/data_jo/001.csv
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper transition bandwidth: 1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper transition bandwidth: 1

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper transition bandwidth: 1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper transition bandwidth: 1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


In [80]:
# Test one PSD feature CSV file
out_file_path = '../data/psd_filtered_features_csv_jo/053.csv'
df = pd.read_csv(out_file_path)
df.head()
df.shape

(20, 144)

### Export all PSD feature CSV files to one CSV file

In [85]:
dir_path = '../data/psd_filtered_features_csv_jo'
# file_names_from_second = ['exp02.csv', 'exp03.csv', 'exp04.csv', 'exp05.csv', 'exp06.csv', 
#               'exp07.csv', 'exp08.csv', 'exp09.csv', 'exp10.csv', 'exp11.csv', 'exp12.csv', 
#               'exp13.csv', 'exp14.csv', 'exp15.csv', 'exp16.csv', 'exp17.csv']
file_path = '../data/psd_filtered_features_csv_jo/001.csv'
all_df = pd.read_csv(file_path)
all_df.head()

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-106.588969,-114.617508,-115.790711,-120.400378,-124.831465,-115.085881,-118.147766,0.921930,-107.168179,-115.243669,...,-118.741500,0.911965,0.001231,0.009129,0.010360,-0.000808,-0.000741,-0.001549,1,0
1,-101.926965,-109.419453,-109.978504,-118.724397,-122.435192,-109.944303,-114.976672,0.897980,-102.368628,-110.619825,...,-115.677659,0.887753,-0.003851,-0.001424,-0.005276,-0.001936,0.001862,-0.000074,1,0
2,-107.996757,-115.468109,-115.472980,-121.788114,-127.757329,-115.739953,-118.236655,0.905936,-108.871279,-115.352090,...,-118.022418,0.906256,-0.000033,0.002070,0.002036,-0.002639,-0.003526,-0.006165,1,0
3,-102.889163,-114.052748,-115.703183,-121.341409,-126.210166,-114.632157,-118.040557,0.908264,-103.933410,-115.161545,...,-118.766204,0.909694,-0.004074,-0.003062,-0.007135,-0.001562,0.004495,0.002933,1,0
4,-109.024912,-117.737440,-117.429492,-124.699051,-130.827770,-117.596563,-122.456928,0.898865,-107.720567,-117.257953,...,-122.530503,0.898941,-0.002029,-0.004111,-0.006140,-0.001638,0.001787,0.000149,1,0


In [86]:
for key in file_name_dict:
    if key == 1:
        continue
    file_name = file_name_dict[key]
    file_path = dir_path + '/' + file_name
    temp_df = pd.read_csv(file_path)
    frames = [all_df, temp_df]
    all_df = pd.concat(frames,ignore_index=True)



# for file_name in file_names_from_second:
#     file_path = dir_path + '/' + file_name
#     temp_df = pd.read_csv(file_path)
#     frames = [all_df, temp_df]
#     all_df = pd.concat(frames,ignore_index=True)

In [87]:
all_df.head(-5)

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-106.588969,-114.617508,-115.790711,-120.400378,-124.831465,-115.085881,-118.147766,0.921930,-107.168179,-115.243669,...,-118.741500,0.911965,0.001231,0.009129,0.010360,-0.000808,-0.000741,-0.001549,1,0
1,-101.926965,-109.419453,-109.978504,-118.724397,-122.435192,-109.944303,-114.976672,0.897980,-102.368628,-110.619825,...,-115.677659,0.887753,-0.003851,-0.001424,-0.005276,-0.001936,0.001862,-0.000074,1,0
2,-107.996757,-115.468109,-115.472980,-121.788114,-127.757329,-115.739953,-118.236655,0.905936,-108.871279,-115.352090,...,-118.022418,0.906256,-0.000033,0.002070,0.002036,-0.002639,-0.003526,-0.006165,1,0
3,-102.889163,-114.052748,-115.703183,-121.341409,-126.210166,-114.632157,-118.040557,0.908264,-103.933410,-115.161545,...,-118.766204,0.909694,-0.004074,-0.003062,-0.007135,-0.001562,0.004495,0.002933,1,0
4,-109.024912,-117.737440,-117.429492,-124.699051,-130.827770,-117.596563,-122.456928,0.898865,-107.720567,-117.257953,...,-122.530503,0.898941,-0.002029,-0.004111,-0.006140,-0.001638,0.001787,0.000149,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,-102.518559,-110.666379,-108.496969,-121.004158,-124.139436,-110.263788,-119.094790,0.888225,-101.154484,-111.014780,...,-118.266238,0.876865,0.004419,0.006281,0.010700,0.001381,0.002891,0.004273,55,0
1091,-102.877493,-111.556695,-106.560125,-120.269276,-122.910081,-109.688180,-117.468708,0.892426,-99.726492,-108.922343,...,-115.636117,0.892073,0.005337,0.007443,0.012781,0.000610,-0.001710,-0.001100,55,0
1092,-103.282182,-110.745569,-111.024376,-120.328627,-124.151483,-111.435891,-117.272241,0.897580,-102.486713,-110.589573,...,-115.932112,0.900104,-0.000381,-0.002007,-0.002387,0.000227,0.000694,0.000921,55,0
1093,-100.312373,-111.624347,-110.411991,-121.023134,-124.477914,-111.662338,-119.546392,0.897045,-100.960867,-109.306861,...,-117.216964,0.881985,-0.001209,0.008240,0.007030,0.000269,0.000600,0.000869,55,0


In [88]:
all_df.shape

(1100, 144)

In [89]:
file_path = '../data/psd_filtered_features_csv_jo/all.csv'
all_df.to_csv(file_path,index=False)

In [90]:
# Test all.csv PSD feature CSV file
file_path = '../data/psd_filtered_features_csv_jo/all.csv'
df = pd.read_csv(file_path)
df.head()

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-106.588969,-114.617508,-115.790711,-120.400378,-124.831465,-115.085881,-118.147766,0.921930,-107.168179,-115.243669,...,-118.741500,0.911965,0.001231,0.009129,0.010360,-0.000808,-0.000741,-0.001549,1,0
1,-101.926965,-109.419453,-109.978504,-118.724397,-122.435192,-109.944303,-114.976672,0.897980,-102.368628,-110.619825,...,-115.677659,0.887753,-0.003851,-0.001424,-0.005276,-0.001936,0.001862,-0.000074,1,0
2,-107.996757,-115.468109,-115.472980,-121.788114,-127.757329,-115.739953,-118.236655,0.905936,-108.871279,-115.352090,...,-118.022418,0.906256,-0.000033,0.002070,0.002036,-0.002639,-0.003526,-0.006165,1,0
3,-102.889163,-114.052748,-115.703183,-121.341409,-126.210166,-114.632157,-118.040557,0.908264,-103.933410,-115.161545,...,-118.766204,0.909694,-0.004074,-0.003062,-0.007135,-0.001562,0.004495,0.002933,1,0
4,-109.024912,-117.737440,-117.429492,-124.699051,-130.827770,-117.596563,-122.456928,0.898865,-107.720567,-117.257953,...,-122.530503,0.898941,-0.002029,-0.004111,-0.006140,-0.001638,0.001787,0.000149,1,0


In [91]:
temp_list = list()
for i in df.head(0):
    temp_list.append(i)

In [41]:
temp_list

['mean_Delta',
 'mean_Theta',
 'mean_Alpha',
 'mean_Beta',
 'mean_Gamma',
 'mean_Slow',
 'mean_Low_beta',
 'mean_Relative_gamma',
 'Fp1_Delta',
 'Fp1_Theta',
 'Fp1_Alpha',
 'Fp1_Beta',
 'Fp1_Gamma',
 'Fp1_Slow',
 'Fp1_Low_beta',
 'Fp1_Relative_gamma',
 'Fp2_Delta',
 'Fp2_Theta',
 'Fp2_Alpha',
 'Fp2_Beta',
 'Fp2_Gamma',
 'Fp2_Slow',
 'Fp2_Low_beta',
 'Fp2_Relative_gamma',
 'F7_Delta',
 'F7_Theta',
 'F7_Alpha',
 'F7_Beta',
 'F7_Gamma',
 'F7_Slow',
 'F7_Low_beta',
 'F7_Relative_gamma',
 'F3_Delta',
 'F3_Theta',
 'F3_Alpha',
 'F3_Beta',
 'F3_Gamma',
 'F3_Slow',
 'F3_Low_beta',
 'F3_Relative_gamma',
 'F4_Delta',
 'F4_Theta',
 'F4_Alpha',
 'F4_Beta',
 'F4_Gamma',
 'F4_Slow',
 'F4_Low_beta',
 'F4_Relative_gamma',
 'F8_Delta',
 'F8_Theta',
 'F8_Alpha',
 'F8_Beta',
 'F8_Gamma',
 'F8_Slow',
 'F8_Low_beta',
 'F8_Relative_gamma',
 'T3_Delta',
 'T3_Theta',
 'T3_Alpha',
 'T3_Beta',
 'T3_Gamma',
 'T3_Slow',
 'T3_Low_beta',
 'T3_Relative_gamma',
 'C3_Delta',
 'C3_Theta',
 'C3_Alpha',
 'C3_Beta',
 'C3_

In [22]:
#cols = df.columns
#cols
#cols[60:150]

In [23]:
#temp_list = ['mean_Delta', 'mean_Theta', 'mean_Alpha', 'mean_Beta', 'mean_Gamma', 'mean_Slow', 'mean_Low_beta', 'Fp1_Delta', 'Fp1_Theta', 'Fp1_Alpha', 'Fp1_Beta', 'Fp1_Gamma', 'Fp1_Slow', 'Fp1_Low_beta', 'Fp2_Delta', 'Fp2_Theta', 'Fp2_Alpha', 'Fp2_Beta', 'Fp2_Gamma', 'Fp2_Slow', 'Fp2_Low_beta', 'F7_Delta', 'F7_Theta', 'F7_Alpha', 'F7_Beta', 'F7_Gamma', 'F7_Slow', 'F7_Low_beta', 'F3_Delta', 'F3_Theta', 'F3_Alpha', 'F3_Beta', 'F3_Gamma', 'F3_Slow', 'F3_Low_beta', 'F4_Delta', 'F4_Theta', 'F4_Alpha', 'F4_Beta', 'F4_Gamma', 'F4_Slow', 'F4_Low_beta', 'F8_Delta', 'F8_Theta', 'F8_Alpha', 'F8_Beta', 'F8_Gamma', 'F8_Slow', 'F8_Low_beta', 'T3_Delta', 'T3_Theta', 'T3_Alpha', 'T3_Beta', 'T3_Gamma', 'T3_Slow', 'T3_Low_beta', 'C3_Delta', 'C3_Theta', 'C3_Alpha', 'C3_Beta', 'C3_Gamma', 'C3_Slow', 'C3_Low_beta', 'C4_Delta', 'C4_Theta', 'C4_Alpha', 'C4_Beta', 'C4_Gamma', 'C4_Slow', 'C4_Low_beta', 'T4_Delta', 'T4_Theta', 'T4_Alpha', 'T4_Beta', 'T4_Gamma', 'T4_Slow', 'T4_Low_beta', 'T5_Delta', 'T5_Theta', 'T5_Alpha', 'T5_Beta', 'T5_Gamma', 'T5_Slow', 'T5_Low_beta', 'P3_Delta', 'P3_Theta', 'P3_Alpha', 'P3_Beta', 'P3_Gamma', 'P3_Slow', 'P3_Low_beta', 'P4_Delta', 'P4_Theta', 'P4_Alpha', 'P4_Beta', 'P4_Gamma', 'P4_Slow', 'P4_Low_beta', 'T6_Delta', 'T6_Theta', 'T6_Alpha', 'T6_Beta', 'T6_Gamma', 'T6_Slow', 'T6_Low_beta', 'O1_Delta', 'O1_Theta', 'O1_Alpha', 'O1_Beta', 'O1_Gamma', 'O1_Slow', 'O1_Low_beta', 'O2_Delta', 'O2_Theta', 'O2_Alpha', 'O2_Beta', 'O2_Gamma', 'O2_Slow', 'O2_Low_beta', 'alpha_frontal', 'alpha_temporal', 'alpha_asymmetry', 'beta_frontal', 'beta_temporal', 'beat_asymmetry', 'participant_id', 'label']

In [92]:
len(temp_list)

144

In [25]:
# a = np.array([[2],[4],[6]])
# a.shape

In [26]:
# b = np.array([[1],[2],[3]])

# a/b